In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
import sys
import joblib

# project root & import path
PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT / "src"))

from config import DATA_PROC, RESULTS_DIR, MODELS_DIR, SEED
from tuning import tune_xgboost, tune_lightgbm

# load features
X_train = load_npz(DATA_PROC / "X_train_tabred_default.npz").toarray()
y_train = np.load(DATA_PROC / "y_train_default.npy")

# ---- 1) XGBoost tuning ----
xgb_best, xgb_cv = tune_xgboost(X_train, y_train, n_iter=30, cv_splits=3)
xgb_cv.to_csv(RESULTS_DIR / "xgb_tuning_results.csv", index=False)
joblib.dump(xgb_best, MODELS_DIR / "xgb_tuned.pkl")

# ---- 2) LightGBM tuning ----
lgb_best, lgb_cv = tune_lightgbm(X_train, y_train, n_iter=30, cv_splits=3)
lgb_cv.to_csv(RESULTS_DIR / "lgbm_tuning_results.csv", index=False)
joblib.dump(lgb_best, MODELS_DIR / "lgbm_tuned.pkl")


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023654 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2434
[LightGBM] [Info] Number of data points in the train set: 19616, number of used features: 300
[LightGBM] [Info] Start training from score 382.867786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

['C:\\Users\\hisuk\\mol-desc-predictor\\results\\model_artifacts\\lgbm_tuned.pkl']

In [ ]:
from train_utils import append_results_table
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
import sys
import joblib

# project root & import path
PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT / "src"))

from config import DATA_PROC, RESULTS_DIR, MODELS_DIR, SEED
from tuning import tune_xgboost, tune_lightgbm

# load features
X_train = load_npz(DATA_PROC / "X_train_tabred_default.npz").toarray()
y_train = np.load(DATA_PROC / "y_train_default.npy")
# ---- 1) XGBoost tuning ----
xgb_best, xgb_cv = tune_xgboost(X_train, y_train, n_iter=30, cv_splits=3)
xgb_cv.to_csv(RESULTS_DIR / "xgb_tuning_results.csv", index=False)
joblib.dump(xgb_best, MODELS_DIR / "xgb_tuned.pkl")

# XGBoost: best row에서 mean / std 추출
best_idx_xgb = xgb_cv["rank_test_score"].idxmin()
xgb_mean_rmse = -xgb_cv.loc[best_idx_xgb, "mean_test_score"]   # neg RMSE → 양수
xgb_std_rmse  =  xgb_cv.loc[best_idx_xgb, "std_test_score"]

xgb_results = {
    "XGBoost_tuned": {
        "mean_RMSE": xgb_mean_rmse,
        "std_RMSE": xgb_std_rmse,
    }
}

df_results = append_results_table(
    results=xgb_results,
    experiment_name="xgb_tuned",
    csv_path=RESULTS_DIR / "metrics_summary.csv",
)

# ---- 2) LightGBM tuning ----
lgb_best, lgb_cv = tune_lightgbm(X_train, y_train, n_iter=30, cv_splits=3)
lgb_cv.to_csv(RESULTS_DIR / "lgbm_tuning_results.csv", index=False)
joblib.dump(lgb_best, MODELS_DIR / "lgbm_tuned.pkl")

best_idx_lgb = lgb_cv["rank_test_score"].idxmin()
lgb_mean_rmse = -lgb_cv.loc[best_idx_lgb, "mean_test_score"]
lgb_std_rmse  =  lgb_cv.loc[best_idx_lgb, "std_test_score"]

lgb_results = {
    "LightGBM_tuned": {
        "mean_RMSE": lgb_mean_rmse,
        "std_RMSE": lgb_std_rmse,
    }
}

df_results = append_results_table(
    results=lgb_results,
    experiment_name="lgbm_tuned",
    csv_path=RESULTS_DIR / "metrics_summary.csv",
)

# 정렬해서 확인
df_results.sort_values("mean_RMSE")



Fitting 3 folds for each of 30 candidates, totalling 90 fits
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015460 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2434
[LightGBM] [Info] Number of data points in the train set: 19616, number of used features: 300
[LightGBM] [Info] Start training from score 382.867786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

,experiment,mean_RMSE,std_RMSE
6,baseline_tabred,41.257918,1.102130
5,baseline_tabred,41.841619,0.953239
4,baseline_tabred,42.662210,1.050662
3,baseline_tabred,43.976861,0.859232
10,lgbm_tuned,46.166548,4.051859
9,xgb_tuned,46.245381,3.999868
8,lgbm_tuned,49.782292,NaN
7,xgb_tuned,52.600526,NaN
2,baseline_tabred,53.523225,0.725667
1,baseline_tabred,53.524891,0.726316
